<a href="https://colab.research.google.com/github/proxima-j/Comp-440-Just-Run/blob/main/COMP_440_Just_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [1]:
!pip install textstat
!pip install transformers
!pip install torch
import textstat
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch
import pandas as pd
import numpy as np
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# mount google dirve
from google.colab import drive
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


# Flesch Scores

## Functions

In [3]:
# Read the aritcle and returns a dataframe containing all the scores and the standard deviation
def get_paragraph_scores(file_path):
    # read article
    with open(file_path, 'r') as f:
        human_news_2015 = f.read()

    # Split paragraphs by two new lines
    human_paragraphs2 = human_news_2015.split('\n\n') 
    human_paragraphs_no_title2 = human_paragraphs2[1:]

    # skip the last paragraph if it is empty
    if not human_paragraphs_no_title2[-1].strip():
        human_paragraphs_no_title2 = human_paragraphs_no_title2[:-1]

    human_scores2 = []
    human_paragraph_order=[]

    for i, paragraph in enumerate(human_paragraphs_no_title2):
        score = textstat.flesch_reading_ease(paragraph)
        human_scores2.append(score)
        human_paragraph_order.append(f"{i+1}")
    
    # Calculate the standard deviation of the scores 
    human_std_dev2 = np.std(human_scores2)
    
    #store the score in a dataframe where scores as row and title as column name
    human_news_flesch_score2 = pd.DataFrame({"Flesch Score":human_scores2})
    human_news_flesch_score2["title"]=human_paragraphs2[0]
    human_news_flesch_score2["Paragraph Order"]=human_paragraph_order
    
    return (human_news_flesch_score2, human_std_dev2)


## News Articles

### News Article 1: 2015 smashes record for hottest year, final figures confirm

In [4]:
# read in the article
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)2015 smashes record for hottest year, final figures confirm.txt'

# returns a dataframe containing all the scores and the standard deviation
df_news_1_human, std_dev = get_paragraph_scores(file_path)

# add the writing prompt, id of the article and generator to the dataset
df_news_1_human["prompt"] = "news"
df_news_1_human["id"] = "n1h"
df_news_1_human["generator"] = "human"

# show the dataset
df_news_1_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,35.61,"2015 smashes record for hottest year, final fi...",1,news,n1h,human
1,53.55,"2015 smashes record for hottest year, final fi...",2,news,n1h,human
2,72.36,"2015 smashes record for hottest year, final fi...",3,news,n1h,human
3,31.55,"2015 smashes record for hottest year, final fi...",4,news,n1h,human
4,52.87,"2015 smashes record for hottest year, final fi...",5,news,n1h,human
5,54.52,"2015 smashes record for hottest year, final fi...",6,news,n1h,human
6,57.61,"2015 smashes record for hottest year, final fi...",7,news,n1h,human
7,40.52,"2015 smashes record for hottest year, final fi...",8,news,n1h,human
8,67.08,"2015 smashes record for hottest year, final fi...",9,news,n1h,human
9,68.40,"2015 smashes record for hottest year, final fi...",10,news,n1h,human


In [5]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)2015 smashes record for hottest year, final figures confirm.txt'
df_news_1_ai, std_dev = get_paragraph_scores(file_path)
df_news_1_ai["prompt"] = "news"
df_news_1_ai["id"] = "n1a"
df_news_1_ai["generator"] = "ai"
df_news_1_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,49.86,"﻿2015 Smashes Record for Hottest Year, Final F...",1,news,n1a,ai
1,46.14,"﻿2015 Smashes Record for Hottest Year, Final F...",2,news,n1a,ai
2,60.18,"﻿2015 Smashes Record for Hottest Year, Final F...",3,news,n1a,ai
3,50.36,"﻿2015 Smashes Record for Hottest Year, Final F...",4,news,n1a,ai
4,37.84,"﻿2015 Smashes Record for Hottest Year, Final F...",5,news,n1a,ai
5,57.10,"﻿2015 Smashes Record for Hottest Year, Final F...",6,news,n1a,ai


### News Article 2: Bigger Chickens Bring a Tough New Problem_ ‘Woody Breast’

In [6]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)Bigger Chickens Bring a Tough New Problem_ ‘Woody Breast’.txt'
df_news_2_human, std_dev = get_paragraph_scores(file_path)
df_news_2_human["prompt"] = "news"
df_news_2_human["id"] = "n2h"
df_news_2_human["generator"] = "human"
df_news_2_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,57.61,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,1,news,n2h,human
1,78.79,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,2,news,n2h,human
2,71.85,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,3,news,n2h,human
3,67.08,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,4,news,n2h,human
4,74.49,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,5,news,n2h,human
5,59.64,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,6,news,n2h,human
6,64.41,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,7,news,n2h,human
7,48.47,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,8,news,n2h,human
8,71.14,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,9,news,n2h,human
9,65.56,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,10,news,n2h,human


In [7]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Bigger Chickens Bring a Tough New Problem_ ‘Woody Breast’.txt'
df_news_2_ai, std_dev = get_paragraph_scores(file_path)
df_news_2_ai["prompt"] = "news"
df_news_2_ai["id"] = "n2a"
df_news_2_ai["generator"] = "ai"
df_news_2_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,63.73,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,1,news,n2a,ai
1,62.38,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,2,news,n2a,ai
2,52.53,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,3,news,n2a,ai
3,56.29,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,4,news,n2a,ai
4,61.97,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,5,news,n2a,ai
5,41.40,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,6,news,n2a,ai
6,40.69,﻿Bigger Chickens Bring a Tough New Problem: ‘W...,7,news,n2a,ai


### News Article 3: F1 circuits could withdraw from rescheduled 2020 calendar due to fallout from coronavirus pandemic

In [8]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)F1 circuits could withdraw from rescheduled 2020 calendar due to fallout from coronavirus pandemic.txt'
df_news_3_human, std_dev = get_paragraph_scores(file_path)
df_news_3_human["prompt"] = "news"
df_news_3_human["id"] = "n3h"
df_news_3_human["generator"] = "human"
df_news_3_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,52.87,﻿F1 circuits could withdraw from rescheduled 2...,1,news,n3h,human
1,42.38,﻿F1 circuits could withdraw from rescheduled 2...,2,news,n3h,human
2,65.73,﻿F1 circuits could withdraw from rescheduled 2...,3,news,n3h,human
3,41.71,﻿F1 circuits could withdraw from rescheduled 2...,4,news,n3h,human
4,85.02,﻿F1 circuits could withdraw from rescheduled 2...,5,news,n3h,human
5,77.91,﻿F1 circuits could withdraw from rescheduled 2...,6,news,n3h,human
6,80.62,﻿F1 circuits could withdraw from rescheduled 2...,7,news,n3h,human
7,37.98,﻿F1 circuits could withdraw from rescheduled 2...,8,news,n3h,human
8,46.44,﻿F1 circuits could withdraw from rescheduled 2...,9,news,n3h,human
9,33.25,﻿F1 circuits could withdraw from rescheduled 2...,10,news,n3h,human


In [9]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)F1 circuits could withdraw from rescheduled 2020 calendar due to fallout from coronavirus pandemic.txt'
df_news_3_ai, std_dev = get_paragraph_scores(file_path)
df_news_3_ai["prompt"] = "news"
df_news_3_ai["id"] = "n3a"
df_news_3_ai["generator"] = "ai"
df_news_3_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,46.95,﻿F1 circuits could withdraw from rescheduled 2...,1,news,n3a,ai
1,57.61,﻿F1 circuits could withdraw from rescheduled 2...,2,news,n3a,ai
2,70.13,﻿F1 circuits could withdraw from rescheduled 2...,3,news,n3a,ai
3,56.08,﻿F1 circuits could withdraw from rescheduled 2...,4,news,n3a,ai
4,67.79,﻿F1 circuits could withdraw from rescheduled 2...,5,news,n3a,ai
5,60.99,﻿F1 circuits could withdraw from rescheduled 2...,6,news,n3a,ai
6,46.81,﻿F1 circuits could withdraw from rescheduled 2...,7,news,n3a,ai


### News Article 4: Richard Scudamore says Premier League is _not a charity_ following £5bn TV deal

In [10]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)News Article 4.txt'
df_news_4_human, std_dev = get_paragraph_scores(file_path)
df_news_4_human["prompt"] = "news"
df_news_4_human["id"] = "n4h"
df_news_4_human["generator"] = "human"
df_news_4_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,47.46,Richard Scudamore says Premier League is 'not ...,1,news,n4h,human
1,56.25,Richard Scudamore says Premier League is 'not ...,2,news,n4h,human
2,34.94,Richard Scudamore says Premier League is 'not ...,3,news,n4h,human
3,56.26,Richard Scudamore says Premier League is 'not ...,4,news,n4h,human
4,90.46,Richard Scudamore says Premier League is 'not ...,5,news,n4h,human
5,46.44,Richard Scudamore says Premier League is 'not ...,6,news,n4h,human
6,35.95,Richard Scudamore says Premier League is 'not ...,7,news,n4h,human
7,72.66,Richard Scudamore says Premier League is 'not ...,8,news,n4h,human
8,68.44,Richard Scudamore says Premier League is 'not ...,9,news,n4h,human
9,82.99,Richard Scudamore says Premier League is 'not ...,10,news,n4h,human


In [11]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Richard Scudamore says Premier League is _not a charity_ following £5bn TV deal.txt'
df_news_4_ai, std_dev = get_paragraph_scores(file_path)
df_news_4_ai["prompt"] = "news"
df_news_4_ai["id"] = "n4a"
df_news_4_ai["generator"] = "ai"
df_news_4_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,59.64,﻿Richard Scudamore says Premier League is 'not...,1,news,n4a,ai
1,86.30,﻿Richard Scudamore says Premier League is 'not...,2,news,n4a,ai
2,74.53,﻿Richard Scudamore says Premier League is 'not...,3,news,n4a,ai
3,61.67,﻿Richard Scudamore says Premier League is 'not...,4,news,n4a,ai
4,53.55,﻿Richard Scudamore says Premier League is 'not...,5,news,n4a,ai
5,68.40,﻿Richard Scudamore says Premier League is 'not...,6,news,n4a,ai
6,44.58,﻿Richard Scudamore says Premier League is 'not...,7,news,n4a,ai
7,63.02,﻿Richard Scudamore says Premier League is 'not...,8,news,n4a,ai
8,59.98,﻿Richard Scudamore says Premier League is 'not...,9,news,n4a,ai


### News Article 5: Stocks Fall, Wiping Out Gains for 2018

In [12]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)Stocks Fall, Wiping Out Gains for 2018.txt'
df_news_5_human, std_dev = get_paragraph_scores(file_path)
df_news_5_human["prompt"] = "news"
df_news_5_human["id"] = "n5h"
df_news_5_human["generator"] = "human"
df_news_5_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,71.65,"Stocks Fall, Wiping Out Gains for 2018",1,news,n5h,human
1,41.53,"Stocks Fall, Wiping Out Gains for 2018",2,news,n5h,human
2,49.65,"Stocks Fall, Wiping Out Gains for 2018",3,news,n5h,human
3,18.70,"Stocks Fall, Wiping Out Gains for 2018",4,news,n5h,human
4,70.84,"Stocks Fall, Wiping Out Gains for 2018",5,news,n5h,human
5,38.32,"Stocks Fall, Wiping Out Gains for 2018",6,news,n5h,human
6,57.95,"Stocks Fall, Wiping Out Gains for 2018",7,news,n5h,human
7,57.30,"Stocks Fall, Wiping Out Gains for 2018",8,news,n5h,human
8,69.79,"Stocks Fall, Wiping Out Gains for 2018",9,news,n5h,human
9,63.53,"Stocks Fall, Wiping Out Gains for 2018",10,news,n5h,human


In [13]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Stocks Fall, Wiping Out Gains for 2018.txt'
df_news_5_ai, std_dev = get_paragraph_scores(file_path)
df_news_5_ai["prompt"] = "news"
df_news_5_ai["id"] = "n5a"
df_news_5_ai["generator"] = "ai"
df_news_5_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,64.24,"﻿Stocks Fall, Wiping Out Gains for 2018",1,news,n5a,ai
1,60.28,"﻿Stocks Fall, Wiping Out Gains for 2018",2,news,n5a,ai
2,61.33,"﻿Stocks Fall, Wiping Out Gains for 2018",3,news,n5a,ai
3,59.98,"﻿Stocks Fall, Wiping Out Gains for 2018",4,news,n5a,ai
4,51.68,"﻿Stocks Fall, Wiping Out Gains for 2018",5,news,n5a,ai
5,53.24,"﻿Stocks Fall, Wiping Out Gains for 2018",6,news,n5a,ai


## Medium Blog Post Articles

### Medium Blog Post 1： AR _ VR _ MR Technology Trends Review

In [14]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)AR _ VR _ MR Technology Trends Review.txt'
df_blog_1_human, std_dev = get_paragraph_scores(file_path)
df_blog_1_human["prompt"] = "blog"
df_blog_1_human["id"] = "b1h"
df_blog_1_human["generator"] = "human"
df_blog_1_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,67.08,﻿AR & VR & MR Technology Trends Review,1,blog,b1h,human
1,43.06,﻿AR & VR & MR Technology Trends Review,2,blog,b1h,human
2,46.10,﻿AR & VR & MR Technology Trends Review,3,blog,b1h,human
3,61.16,﻿AR & VR & MR Technology Trends Review,4,blog,b1h,human
4,44.75,﻿AR & VR & MR Technology Trends Review,5,blog,b1h,human
5,79.09,﻿AR & VR & MR Technology Trends Review,6,blog,b1h,human
6,49.01,﻿AR & VR & MR Technology Trends Review,7,blog,b1h,human
7,65.42,﻿AR & VR & MR Technology Trends Review,8,blog,b1h,human
8,58.58,﻿AR & VR & MR Technology Trends Review,9,blog,b1h,human
9,65.52,﻿AR & VR & MR Technology Trends Review,10,blog,b1h,human


In [15]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)AR _ VR _ MR Technology Trends Review.txt'
df_blog_1_ai, std_dev = get_paragraph_scores(file_path)
df_blog_1_ai["prompt"] = "blog"
df_blog_1_ai["id"] = "b1a"
df_blog_1_ai["generator"] = "ai"
df_blog_1_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,36.28,AR & VR & MR Technology Trends Review,1,blog,b1a,ai
1,56.45,AR & VR & MR Technology Trends Review,2,blog,b1a,ai
2,42.41,AR & VR & MR Technology Trends Review,3,blog,b1a,ai
3,49.65,AR & VR & MR Technology Trends Review,4,blog,b1a,ai
4,50.16,AR & VR & MR Technology Trends Review,5,blog,b1a,ai


### Medium Blog Post 2： Blockchain Bling_ How NFTs Are Revolutionizing The Luxury Industry (outlier!!!)

In [16]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)Blockchain Bling_ How NFTs Are Revolutionizing The Luxury Industry.txt'
df_blog_2_human, std_dev = get_paragraph_scores(file_path)
df_blog_2_human["prompt"] = "blog"
df_blog_2_human["id"] = "b2h"
df_blog_2_human["generator"] = "human"
df_blog_2_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,46.40,Blockchain Bling: How NFTs Are Revolutionizing...,1,blog,b2h,human
1,32.22,Blockchain Bling: How NFTs Are Revolutionizing...,2,blog,b2h,human
2,45.29,Blockchain Bling: How NFTs Are Revolutionizing...,3,blog,b2h,human
3,53.55,Blockchain Bling: How NFTs Are Revolutionizing...,4,blog,b2h,human
4,56.25,Blockchain Bling: How NFTs Are Revolutionizing...,5,blog,b2h,human
5,40.89,Blockchain Bling: How NFTs Are Revolutionizing...,6,blog,b2h,human
6,25.73,Blockchain Bling: How NFTs Are Revolutionizing...,7,blog,b2h,human
7,37.94,Blockchain Bling: How NFTs Are Revolutionizing...,8,blog,b2h,human
8,39.37,Blockchain Bling: How NFTs Are Revolutionizing...,9,blog,b2h,human
9,44.75,Blockchain Bling: How NFTs Are Revolutionizing...,10,blog,b2h,human


In [17]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)Blockchain Bling_ How NFTs Are Revolutionizing The Luxury Industry.txt'
df_blog_2_ai, std_dev = get_paragraph_scores(file_path)
df_blog_2_ai["prompt"] = "blog"
df_blog_2_ai["id"] = "b2a"
df_blog_2_ai["generator"] = "ai"
df_blog_2_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,38.15,﻿Blockchain Bling: How NFTs Are Revolutionizin...,1,blog,b2a,ai
1,54.42,﻿Blockchain Bling: How NFTs Are Revolutionizin...,2,blog,b2a,ai
2,56.59,﻿Blockchain Bling: How NFTs Are Revolutionizin...,3,blog,b2a,ai
3,63.90,﻿Blockchain Bling: How NFTs Are Revolutionizin...,4,blog,b2a,ai
4,53.41,﻿Blockchain Bling: How NFTs Are Revolutionizin...,5,blog,b2a,ai
5,23.05,﻿Blockchain Bling: How NFTs Are Revolutionizin...,6,blog,b2a,ai
6,19.06,﻿Blockchain Bling: How NFTs Are Revolutionizin...,7,blog,b2a,ai


### Medium Blog Post 3： How Will the CryptoFlix Film Market Place Work

In [18]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)How Will the CryptoFlix Film Market Place Work_.txt'
df_blog_3_human, std_dev = get_paragraph_scores(file_path)
df_blog_3_human["prompt"] = "blog"
df_blog_3_human["id"] = "b3h"
df_blog_3_human["generator"] = "human"
df_blog_3_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,67.42,﻿How Will the CryptoFlix Film Market Place Work?,1,blog,b3h,human
1,57.81,﻿How Will the CryptoFlix Film Market Place Work?,2,blog,b3h,human
2,30.20,﻿How Will the CryptoFlix Film Market Place Work?,3,blog,b3h,human
3,78.25,﻿How Will the CryptoFlix Film Market Place Work?,4,blog,b3h,human
4,75.71,﻿How Will the CryptoFlix Film Market Place Work?,5,blog,b3h,human
5,49.49,﻿How Will the CryptoFlix Film Market Place Work?,6,blog,b3h,human
6,36.96,﻿How Will the CryptoFlix Film Market Place Work?,7,blog,b3h,human
7,42.04,﻿How Will the CryptoFlix Film Market Place Work?,8,blog,b3h,human
8,121.22,﻿How Will the CryptoFlix Film Market Place Work?,9,blog,b3h,human
9,51.52,﻿How Will the CryptoFlix Film Market Place Work?,10,blog,b3h,human


In [19]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)How Will the CryptoFlix Film Market Place Work_.txt'
df_blog_3_ai, std_dev = get_paragraph_scores(file_path)
df_blog_3_ai["prompt"] = "blog"
df_blog_3_ai["id"] = "b3a"
df_blog_3_ai["generator"] = "ai"
df_blog_3_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,54.42,﻿How Will the CryptoFlix Film Market Place Work?,1,blog,b3a,ai
1,60.65,﻿How Will the CryptoFlix Film Market Place Work?,2,blog,b3a,ai
2,49.45,﻿How Will the CryptoFlix Film Market Place Work?,3,blog,b3a,ai
3,45.09,﻿How Will the CryptoFlix Film Market Place Work?,4,blog,b3a,ai
4,60.65,﻿How Will the CryptoFlix Film Market Place Work?,5,blog,b3a,ai
5,61.97,﻿How Will the CryptoFlix Film Market Place Work?,6,blog,b3a,ai
6,40.72,﻿How Will the CryptoFlix Film Market Place Work?,7,blog,b3a,ai


### Medium Blog Post 4： The upside of a downturn

In [20]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)The upside of a downturn_.txt'
df_blog_4_human, std_dev = get_paragraph_scores(file_path)
df_blog_4_human["prompt"] = "blog"
df_blog_4_human["id"] = "b4h"
df_blog_4_human["generator"] = "human"
df_blog_4_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,78.25,The upside of a downturn.,1,blog,b4h,human
1,63.39,The upside of a downturn.,2,blog,b4h,human
2,54.63,The upside of a downturn.,3,blog,b4h,human
3,55.24,The upside of a downturn.,4,blog,b4h,human
4,39.50,The upside of a downturn.,5,blog,b4h,human
5,53.37,The upside of a downturn.,6,blog,b4h,human
6,75.20,The upside of a downturn.,7,blog,b4h,human
7,74.19,The upside of a downturn.,8,blog,b4h,human
8,60.35,The upside of a downturn.,9,blog,b4h,human
9,35.98,The upside of a downturn.,10,blog,b4h,human


In [21]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)The upside of a downturn_.txt'
df_blog_4_ai, std_dev = get_paragraph_scores(file_path)
df_blog_4_ai["prompt"] = "blog"
df_blog_4_ai["id"] = "b4a"
df_blog_4_ai["generator"] = "ai"
df_blog_4_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,64.20,﻿The Upside of a Downturn,1,blog,b4a,ai
1,59.94,﻿The Upside of a Downturn,2,blog,b4a,ai
2,67.04,﻿The Upside of a Downturn,3,blog,b4a,ai
3,41.70,﻿The Upside of a Downturn,4,blog,b4a,ai
4,39.53,﻿The Upside of a Downturn,5,blog,b4a,ai
5,44.44,﻿The Upside of a Downturn,6,blog,b4a,ai
6,37.30,﻿The Upside of a Downturn,7,blog,b4a,ai
7,33.58,﻿The Upside of a Downturn,8,blog,b4a,ai


### Medium Blog Post 5： Travel Highlights in Portugal_ Lisbon to Porto

In [22]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)Travel Highlights in Portugal_ Lisbon to Porto.txt'
df_blog_5_human, std_dev = get_paragraph_scores(file_path)
df_blog_5_human["prompt"] = "blog"
df_blog_5_human["id"] = "b5h"
df_blog_5_human["generator"] = "human"
df_blog_5_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,70.13,Travel Highlights in Portugal: Lisbon to Porto,1,blog,b5h,human
1,62.38,Travel Highlights in Portugal: Lisbon to Porto,2,blog,b5h,human
2,63.83,Travel Highlights in Portugal: Lisbon to Porto,3,blog,b5h,human
3,60.04,Travel Highlights in Portugal: Lisbon to Porto,4,blog,b5h,human
4,52.23,Travel Highlights in Portugal: Lisbon to Porto,5,blog,b5h,human
5,56.29,Travel Highlights in Portugal: Lisbon to Porto,6,blog,b5h,human
6,61.70,Travel Highlights in Portugal: Lisbon to Porto,7,blog,b5h,human
7,80.41,Travel Highlights in Portugal: Lisbon to Porto,8,blog,b5h,human
8,71.85,Travel Highlights in Portugal: Lisbon to Porto,9,blog,b5h,human
9,71.95,Travel Highlights in Portugal: Lisbon to Porto,10,blog,b5h,human


In [23]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)Travel Highlights in Portugal_ Lisbon to Porto.txt'
df_blog_5_ai, std_dev = get_paragraph_scores(file_path)
df_blog_5_ai["prompt"] = "blog"
df_blog_5_ai["id"] = "b5a"
df_blog_5_ai["generator"] = "ai"
df_blog_5_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,57.91,Travel Highlights in Portugal: Lisbon to Porto,1,blog,b5a,ai
1,73.17,Travel Highlights in Portugal: Lisbon to Porto,2,blog,b5a,ai
2,54.22,Travel Highlights in Portugal: Lisbon to Porto,3,blog,b5a,ai
3,52.70,Travel Highlights in Portugal: Lisbon to Porto,4,blog,b5a,ai
4,45.05,Travel Highlights in Portugal: Lisbon to Porto,5,blog,b5a,ai
5,44.03,Travel Highlights in Portugal: Lisbon to Porto,6,blog,b5a,ai
6,54.73,Travel Highlights in Portugal: Lisbon to Porto,7,blog,b5a,ai
7,78.59,Travel Highlights in Portugal: Lisbon to Porto,8,blog,b5a,ai
8,66.44,Travel Highlights in Portugal: Lisbon to Porto,9,blog,b5a,ai
9,72.87,Travel Highlights in Portugal: Lisbon to Porto,10,blog,b5a,ai


## Language Proficiency Exam Articles

### LPE 1: A person you know is planning to move to your town or city

In [24]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)A person you know is planning to move to your town or city._.txt'
df_exam_1_human, std_dev = get_paragraph_scores(file_path)
df_exam_1_human["prompt"] = "exam"
df_exam_1_human["id"] = "e1h"
df_exam_1_human["generator"] = "human"
df_exam_1_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,86.03,A person you know is planning to move to your ...,1,exam,e1h,human
1,54.63,A person you know is planning to move to your ...,2,exam,e1h,human
2,81.22,A person you know is planning to move to your ...,3,exam,e1h,human
3,90.90,A person you know is planning to move to your ...,4,exam,e1h,human
4,74.69,A person you know is planning to move to your ...,5,exam,e1h,human


In [25]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)A person you know is planning to move to your town or city._.txt'
df_exam_1_ai, std_dev = get_paragraph_scores(file_path)
df_exam_1_ai["prompt"] = "exam"
df_exam_1_ai["id"] = "e1a"
df_exam_1_ai["generator"] = "ai"
df_exam_1_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,73.51,﻿A person you know is planning to move to your...,1,exam,e1a,ai
1,51.89,﻿A person you know is planning to move to your...,2,exam,e1a,ai
2,60.35,﻿A person you know is planning to move to your...,3,exam,e1a,ai
3,63.29,﻿A person you know is planning to move to your...,4,exam,e1a,ai
4,47.49,﻿A person you know is planning to move to your...,5,exam,e1a,ai
5,56.08,﻿A person you know is planning to move to your...,6,exam,e1a,ai


### LPE 2: Businesses should hire employees for their entire lives

In [26]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Businesses should hire employees for their entire lives._.txt'
df_exam_2_human, std_dev = get_paragraph_scores(file_path)
df_exam_2_human["prompt"] = "exam"
df_exam_2_human["id"] = "e2h"
df_exam_2_human["generator"] = "human"
df_exam_2_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,44.27,Businesses should hire employees for their ent...,1,exam,e2h,human
1,49.15,Businesses should hire employees for their ent...,2,exam,e2h,human
2,54.52,Businesses should hire employees for their ent...,3,exam,e2h,human
3,66.07,Businesses should hire employees for their ent...,4,exam,e2h,human
4,53.55,Businesses should hire employees for their ent...,5,exam,e2h,human


In [27]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Businesses should hire employees for their entire lives_.txt'
df_exam_2_ai, std_dev = get_paragraph_scores(file_path)
df_exam_2_ai["prompt"] = "exam"
df_exam_2_ai["id"] = "e2a"
df_exam_2_ai["generator"] = "ai"
df_exam_2_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,45.76,﻿Businesses should hire employees for their en...,1,exam,e2a,ai
1,42.21,﻿Businesses should hire employees for their en...,2,exam,e2a,ai
2,51.38,﻿Businesses should hire employees for their en...,3,exam,e2a,ai
3,50.16,﻿Businesses should hire employees for their en...,4,exam,e2a,ai
4,36.28,﻿Businesses should hire employees for their en...,5,exam,e2a,ai


### LPE 3: Do you agree or disagree with the following statement_ Telephones

In [28]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Do you agree or disagree with the following statement_ Telephones.txt'
df_exam_3_human, std_dev = get_paragraph_scores(file_path)
df_exam_3_human["prompt"] = "exam"
df_exam_3_human["id"] = "e3h"
df_exam_3_human["generator"] = "human"
df_exam_3_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,44.44,Do you agree or disagree with the following st...,1,exam,e3h,human
1,57.91,Do you agree or disagree with the following st...,2,exam,e3h,human
2,67.28,Do you agree or disagree with the following st...,3,exam,e3h,human
3,38.32,Do you agree or disagree with the following st...,4,exam,e3h,human


In [29]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Do you agree or disagree with the following statement_ Telephones.txt'
df_exam_3_ai, std_dev = get_paragraph_scores(file_path)
df_exam_3_ai["prompt"] = "exam"
df_exam_3_ai["id"] = "e3a"
df_exam_3_ai["generator"] = "ai"
df_exam_3_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,46.78,﻿Do you agree or disagree with the following s...,1,exam,e3a,ai
1,60.14,﻿Do you agree or disagree with the following s...,2,exam,e3a,ai
2,24.48,﻿Do you agree or disagree with the following s...,3,exam,e3a,ai
3,33.24,﻿Do you agree or disagree with the following s...,4,exam,e3a,ai
4,45.46,﻿Do you agree or disagree with the following s...,5,exam,e3a,ai
5,32.26,﻿Do you agree or disagree with the following s...,6,exam,e3a,ai


### LPE 4: Do you agree or disagree with the following statement_ Telephones

In [30]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Some people believe that success in life comes from taking risks or chances._.txt'
df_exam_4_human, std_dev = get_paragraph_scores(file_path)
df_exam_4_human["prompt"] = "exam"
df_exam_4_human["id"] = "e4h"
df_exam_4_human["generator"] = "human"
df_exam_4_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,67.25,Some people believe that success in life comes...,1,exam,e4h,human
1,61.26,Some people believe that success in life comes...,2,exam,e4h,human
2,40.35,Some people believe that success in life comes...,3,exam,e4h,human


In [31]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Some people believe that success in life comes from taking risks or chances_.txt'
df_exam_4_ai, std_dev = get_paragraph_scores(file_path)
df_exam_4_ai["prompt"] = "exam"
df_exam_4_ai["id"] = "e4a"
df_exam_4_ai["generator"] = "ai"
df_exam_4_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,62.88,﻿Some people believe that success in life come...,1,exam,e4a,ai
1,48.84,﻿Some people believe that success in life come...,2,exam,e4a,ai
2,39.33,﻿Some people believe that success in life come...,3,exam,e4a,ai
3,46.27,﻿Some people believe that success in life come...,4,exam,e4a,ai
4,43.73,﻿Some people believe that success in life come...,5,exam,e4a,ai


### LPE 5: Do you agree or disagree with the following statement

In [32]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Do you agree or disagree with the following statement_.txt'
df_exam_5_human, std_dev = get_paragraph_scores(file_path)
df_exam_5_human["prompt"] = "exam"
df_exam_5_human["id"] = "e5h"
df_exam_5_human["generator"] = "human"
df_exam_5_human

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,73.68,Do you agree or disagree with the following st...,1,exam,e5h,human
1,79.60,Do you agree or disagree with the following st...,2,exam,e5h,human
2,67.25,Do you agree or disagree with the following st...,3,exam,e5h,human
3,39.84,Do you agree or disagree with the following st...,4,exam,e5h,human


In [33]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Do you agree or disagree with the following statement__.txt'
df_exam_5_ai, std_dev = get_paragraph_scores(file_path)
df_exam_5_ai["prompt"] = "exam"
df_exam_5_ai["id"] = "e5a"
df_exam_5_ai["generator"] = "ai"
df_exam_5_ai

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,53.71,﻿Do you agree or disagree with the following s...,1,exam,e5a,ai
1,55.74,﻿Do you agree or disagree with the following s...,2,exam,e5a,ai
2,35.47,﻿Do you agree or disagree with the following s...,3,exam,e5a,ai
3,36.49,﻿Do you agree or disagree with the following s...,4,exam,e5a,ai
4,50.87,﻿Do you agree or disagree with the following s...,5,exam,e5a,ai


## Creating One Large Dataset

In [34]:
# merge all the human article datasets together
all_articles = pd.concat([df_news_1_human,df_news_2_human,df_news_3_human,df_news_4_human,df_news_5_human,df_news_1_ai,df_news_2_ai,df_news_3_ai,df_news_4_ai,df_news_5_ai,
                          df_blog_1_human,df_blog_2_human,df_blog_3_human,df_blog_4_human,df_blog_5_human,df_blog_1_ai,df_blog_2_ai,df_blog_3_ai,df_blog_4_ai,df_blog_5_ai,
                          df_exam_1_human,df_exam_2_human,df_exam_3_human,df_exam_4_human,df_exam_5_human,df_exam_1_ai,df_exam_2_ai,df_exam_3_ai,df_exam_4_ai,df_exam_5_ai])
all_articles

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,35.61,"2015 smashes record for hottest year, final fi...",1,news,n1h,human
1,53.55,"2015 smashes record for hottest year, final fi...",2,news,n1h,human
2,72.36,"2015 smashes record for hottest year, final fi...",3,news,n1h,human
3,31.55,"2015 smashes record for hottest year, final fi...",4,news,n1h,human
4,52.87,"2015 smashes record for hottest year, final fi...",5,news,n1h,human
...,...,...,...,...,...,...
0,53.71,﻿Do you agree or disagree with the following s...,1,exam,e5a,ai
1,55.74,﻿Do you agree or disagree with the following s...,2,exam,e5a,ai
2,35.47,﻿Do you agree or disagree with the following s...,3,exam,e5a,ai
3,36.49,﻿Do you agree or disagree with the following s...,4,exam,e5a,ai


In [35]:
# save the dataset into a csv file
all_articles.to_csv('all_articles.csv', index=False)

In [36]:
# download the saved csv file
#from google.colab import files
#files.download('all_articles.csv')

## Creating the Metrics Dataset 

### Standard Deviation

In [37]:
# group the data by id and calculate the standard deviation for each article
std_by_article = all_articles.groupby('id')['Flesch Score'].std()

# create a new dataframe to store the standard deviation values
std_df = std_by_article.to_frame().reset_index().rename(columns={'Flesch Score': 'Standard Deviation'})
std_df



,id,Standard Deviation
0,b1a,7.782940
1,b1h,13.126720
2,b2a,17.543865
3,b2h,11.430226
4,b3a,8.415740
5,b3h,27.763417
6,b4a,13.160097
7,b4h,14.264645
8,b5a,10.600409
9,b5h,7.658517


In [38]:
# group the data by id and calculate the mean for each article
mean_by_article = all_articles.groupby('id')['Flesch Score'].mean()

# create a new dataframe to store the mean values
mean_df = mean_by_article.to_frame().reset_index().rename(columns={'Flesch Score': 'Mean'})
mean_df


,id,Mean
0,b1a,46.990000
1,b1h,57.802381
2,b2a,44.082857
3,b2h,45.251429
4,b3a,53.278571
5,b3h,63.396000
6,b4a,48.466250
7,b4h,59.360800
8,b5a,60.661538
9,b5h,66.551765


In [39]:
# group the data by id and calculate the standard deviation for each group
paragrpah_number_by_article = all_articles.groupby('id')['Flesch Score'].nunique()

# create a new dataframe to store the mean values
paragraph_number_df = paragrpah_number_by_article.to_frame().reset_index().rename(columns={'Flesch Score': 'Paragraph Number'})
paragraph_number_df

,id,Paragraph Number
0,b1a,5
1,b1h,19
2,b2a,7
3,b2h,14
4,b3a,6
5,b3h,15
6,b4a,8
7,b4h,22
8,b5a,13
9,b5h,17


In [40]:
# join the three dataset together on id column

metircs = pd.merge(mean_df, std_df, on='id')
articles_metircs = pd.merge(metircs, paragraph_number_df, on='id')
articles_metircs

,id,Mean,Standard Deviation,Paragraph Number
0,b1a,46.990000,7.782940,5
1,b1h,57.802381,13.126720,19
2,b2a,44.082857,17.543865,7
3,b2h,45.251429,11.430226,14
4,b3a,53.278571,8.415740,6
5,b3h,63.396000,27.763417,15
6,b4a,48.466250,13.160097,8
7,b4h,59.360800,14.264645,22
8,b5a,60.661538,10.600409,13
9,b5h,66.551765,7.658517,17


In [41]:
# store the dataset 
articles_metircs.to_csv('articles_metircs.csv', index=False)

In [42]:
#from google.colab import files
#files.download('articles_metircs.csv')

## Reading Dataset

In [43]:
# read the dataset from the stored csv file
all_articles = pd.read_csv('/content/drive/Shareddrives/Comp440/articles txt version/all_articles.csv')
all_articles

,Flesch Score,title,Paragraph Order,prompt,id,generator
0,35.61,"2015 smashes record for hottest year, final fi...",1,news,n1h,human
1,53.55,"2015 smashes record for hottest year, final fi...",2,news,n1h,human
2,72.36,"2015 smashes record for hottest year, final fi...",3,news,n1h,human
3,31.55,"2015 smashes record for hottest year, final fi...",4,news,n1h,human
4,52.87,"2015 smashes record for hottest year, final fi...",5,news,n1h,human
...,...,...,...,...,...,...
307,53.71,﻿Do you agree or disagree with the following s...,1,exam,e5a,ai
308,55.74,﻿Do you agree or disagree with the following s...,2,exam,e5a,ai
309,35.47,﻿Do you agree or disagree with the following s...,3,exam,e5a,ai
310,36.49,﻿Do you agree or disagree with the following s...,4,exam,e5a,ai


In [44]:
# read the dataset from the stored csv file
articles_metircs = pd.read_csv('/content/drive/Shareddrives/Comp440/articles txt version/articles_metircs.csv')
articles_metircs

,id,Mean,Standard Deviation,Paragraph Number
0,b1a,46.990000,7.782940,5
1,b1h,57.802381,13.126720,19
2,b2a,44.082857,17.543865,7
3,b2h,45.251429,11.430226,14
4,b3a,53.278571,8.415740,6
5,b3h,63.396000,27.763417,15
6,b4a,48.466250,13.160097,8
7,b4h,59.360800,14.264645,22
8,b5a,60.661538,10.600409,13
9,b5h,66.551765,7.658517,17


# BERT Embedding

## Functions

In [45]:
# create a list that contains each paragph as the index
def read_paragraph(file_path):
    # read article
    with open(file_path, 'r') as f:
        article = f.read()

    # Split paragraphs by two new lines
    article_paragraphs = article.split('\n\n') 

    # Delete the index that stored the title thus the list start with the first paragraph
    article_paragraphs_no_title = article_paragraphs[1:]

    # skip the last paragraph if it is empty
    if not article_paragraphs_no_title[-1].strip():
        article_paragraphs_no_title = article_paragraphs_no_title[:-1]
    
    return article_paragraphs_no_title

In [46]:
# get the cosine similarity between each pair of paragraphs in the input list that store the paragraph as its index
# returns a Pandas DataFrame that stores the similarity scores for each pair of paragraphs
def get_cs_sim(first_paragraphs):
  similarities = []

  # Get the embeddings for the first paragraph
  for i in range(0, 5):
    input_ids_a = torch.tensor(tokenizer.encode(first_paragraphs[i], add_special_tokens=True)).unsqueeze(0)
    outputs_a = model(input_ids_a)
    first_embedding = outputs_a.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    # Calculate cosine similarity for each paragraph and store results in DataFrame
    for j in range(0, 5):
      input_ids = torch.tensor(tokenizer.encode(first_paragraphs[j], add_special_tokens=True)).unsqueeze(0)
      outputs = model(input_ids)
      embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
      similarity = cosine_similarity(first_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
      similarities.append((i+1, j+1, similarity))

  # Create DataFrame with similarities 
  df = pd.DataFrame(similarities, columns=['Articl1', "Article2", 'Cosine Similarity'])

  return df

### News Articles

#### Read In Articles

In [47]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)2015 smashes record for hottest year, final figures confirm.txt'
n1a = read_paragraph(file_path)
path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)2015 smashes record for hottest year, final figures confirm.txt'
n1a = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Bigger Chickens Bring a Tough New Problem_ ‘Woody Breast’.txt'
n2a = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)F1 circuits could withdraw from rescheduled 2020 calendar due to fallout from coronavirus pandemic.txt'
n3a = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Richard Scudamore says Premier League is _not a charity_ following £5bn TV deal.txt'
n4a = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(AI)Stocks Fall, Wiping Out Gains for 2018.txt'
n5a = read_paragraph(file_path)

In [48]:
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)2015 smashes record for hottest year, final figures confirm.txt'
n1h = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)Bigger Chickens Bring a Tough New Problem_ ‘Woody Breast’.txt'
n2h = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)F1 circuits could withdraw from rescheduled 2020 calendar due to fallout from coronavirus pandemic.txt'
n3h = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)News Article 4.txt'
n4h = read_paragraph(file_path)
file_path = '/content/drive/Shareddrives/Comp440/articles txt version/News/(Human)Stocks Fall, Wiping Out Gains for 2018.txt'
n5h = read_paragraph(file_path)

#### First Paragraph

##### Store Paragraphs to List

In [49]:
first_paragraphs_h_n = [n1h[0],n2h[0],n3h[0],n4h[0],n5h[0]]

In [50]:
first_paragraphs_a_n = [n1a[0],n2a[0],n3a[0],n4a[0],n5a[0]]

##### Creating Cosine Similarity Dataset

In [51]:
df_n_first_h=get_cs_sim(first_paragraphs_h_n)

In [52]:
# add generator and writng prompt column to the dataset
df_n_first_h["generator"] = "human"
df_n_first_h["prompt"] = "news"

In [53]:
df_n_first_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,news
1,1,2,0.634030,human,news
2,1,3,0.668836,human,news
3,1,4,0.618219,human,news
4,1,5,0.659784,human,news
5,2,1,0.634030,human,news
6,2,2,1.000000,human,news
7,2,3,0.752626,human,news
8,2,4,0.687159,human,news
9,2,5,0.687113,human,news


In [54]:
df_n_first_a=get_cs_sim(first_paragraphs_a_n)

In [55]:
df_n_first_a["generator"] = "ai"
df_n_first_a["prompt"] = "news"

In [56]:
df_n_first_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,news
1,1,2,0.781538,ai,news
2,1,3,0.768003,ai,news
3,1,4,0.730322,ai,news
4,1,5,0.797342,ai,news
5,2,1,0.781538,ai,news
6,2,2,1.000000,ai,news
7,2,3,0.782440,ai,news
8,2,4,0.741517,ai,news
9,2,5,0.802438,ai,news


#### Last Paragraph

##### Store Paragraphs to List

In [57]:
last_paragraph_a_n = [n1a[len(n1a)-1],n2a[len(n2a)-1],n3a[len(n3a)-1],n4a[len(n4a)-1],n5a[len(n5a)-1]]
last_paragraph_h_n = [n1h[len(n1h)-1],n2h[len(n2h)-1],n3h[len(n3h)-1],n4h[len(n4h)-1],n5h[len(n5h)-1]]

##### Creating Cosine Similarity Dataset

In [58]:
df_n_last_h=get_cs_sim(last_paragraph_h_n)

In [59]:
df_n_last_h["generator"] = "human"
df_n_last_h["prompt"] = "news"

In [60]:
df_n_last_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,news
1,1,2,0.825249,human,news
2,1,3,0.747916,human,news
3,1,4,0.752782,human,news
4,1,5,0.572435,human,news
5,2,1,0.825249,human,news
6,2,2,1.000000,human,news
7,2,3,0.751814,human,news
8,2,4,0.738328,human,news
9,2,5,0.489363,human,news


In [61]:
df_n_last_a=get_cs_sim(last_paragraph_a_n)

In [62]:
df_n_last_a["generator"] = "ai"
df_n_last_a["prompt"] = "news"

In [63]:
df_n_last_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,news
1,1,2,0.849937,ai,news
2,1,3,0.870404,ai,news
3,1,4,0.823981,ai,news
4,1,5,0.863704,ai,news
5,2,1,0.849937,ai,news
6,2,2,1.000000,ai,news
7,2,3,0.857068,ai,news
8,2,4,0.822199,ai,news
9,2,5,0.861856,ai,news


### Medium Blog Post Articles

#### Read in Articles

In [64]:
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)AR _ VR _ MR Technology Trends Review.txt"
b1a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)Blockchain Bling_ How NFTs Are Revolutionizing The Luxury Industry.txt"
b2a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)How Will the CryptoFlix Film Market Place Work_.txt"
b3a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)The upside of a downturn_.txt"
b4a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(AI)Travel Highlights in Portugal_ Lisbon to Porto.txt"
b5a = read_paragraph(file_path)

In [65]:
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)AR _ VR _ MR Technology Trends Review.txt"
b1h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)Blockchain Bling_ How NFTs Are Revolutionizing The Luxury Industry.txt"
b2h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)How Will the CryptoFlix Film Market Place Work_.txt"
b3h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)The upside of a downturn_.txt"
b4h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Medium Blog Post/(Human)Travel Highlights in Portugal_ Lisbon to Porto.txt"
b5h = read_paragraph(file_path)

#### First Paragraph

##### Store Paragraphs to List

In [66]:
first_paragraphs_a_b = [b1a[0],b2a[0],b3a[0],b4a[0],b5a[0]]

In [67]:
first_paragraphs_h_b = [b1h[0],b2h[0],b3h[0],b4h[0],b5h[0]]

##### Creating Cosine Similarity Dataset

In [68]:
df_b_first_a=get_cs_sim(first_paragraphs_a_b)

In [69]:
df_b_first_a["generator"] = "ai"
df_b_first_a["prompt"] = "blog"

In [70]:
df_b_first_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,blog
1,1,2,0.834554,ai,blog
2,1,3,0.845976,ai,blog
3,1,4,0.813050,ai,blog
4,1,5,0.770009,ai,blog
5,2,1,0.834554,ai,blog
6,2,2,1.000000,ai,blog
7,2,3,0.907652,ai,blog
8,2,4,0.851435,ai,blog
9,2,5,0.816806,ai,blog


In [71]:
df_b_first_h=get_cs_sim(first_paragraphs_h_b)

In [72]:
df_b_first_h["generator"] = "human"
df_b_first_h["prompt"] = "blog"

In [73]:
df_b_first_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,blog
1,1,2,0.813157,human,blog
2,1,3,0.867873,human,blog
3,1,4,0.703070,human,blog
4,1,5,0.838939,human,blog
5,2,1,0.813157,human,blog
6,2,2,1.000000,human,blog
7,2,3,0.802868,human,blog
8,2,4,0.688363,human,blog
9,2,5,0.804054,human,blog


#### Last Paragraph

##### Store Paragraphs to List

In [74]:
last_paragraphs_a_b = [b1a[len(b1a)-1],b2a[len(b2a)-1],b3a[len(b3a)-1],b4a[len(b4a)-1],b5a[len(b5a)-1]]

In [75]:
last_paragraphs_h_b = [b1h[len(b1h)-1],b2h[len(b2h)-1],b3h[len(b3h)-1],b4h[len(b4h)-1],b5h[len(b5h)-1]]

##### Creating Cosine Similarity Dataset

In [76]:
df_b_last_a=get_cs_sim(last_paragraphs_a_b)

In [77]:
df_b_last_a["generator"] = "ai"
df_b_last_a["prompt"] = "blog"

In [78]:
df_b_last_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,blog
1,1,2,0.873298,ai,blog
2,1,3,0.809654,ai,blog
3,1,4,0.855364,ai,blog
4,1,5,0.731910,ai,blog
5,2,1,0.873298,ai,blog
6,2,2,1.000000,ai,blog
7,2,3,0.860026,ai,blog
8,2,4,0.920007,ai,blog
9,2,5,0.732008,ai,blog


In [79]:
df_b_last_h=get_cs_sim(last_paragraphs_h_b)

In [80]:
df_b_last_h["generator"] = "human"
df_b_last_h["prompt"] = "blog"

In [81]:
df_b_last_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,blog
1,1,2,0.866440,human,blog
2,1,3,0.449033,human,blog
3,1,4,0.784092,human,blog
4,1,5,0.783430,human,blog
5,2,1,0.866440,human,blog
6,2,2,1.000000,human,blog
7,2,3,0.409237,human,blog
8,2,4,0.812144,human,blog
9,2,5,0.778998,human,blog


### Language Proficiency Exam Articles

#### Read in Articles

In [82]:
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)A person you know is planning to move to your town or city._.txt"
e1a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Businesses should hire employees for their entire lives_.txt"
e2a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Do you agree or disagree with the following statement_ Telephones.txt"
e3a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Do you agree or disagree with the following statement__.txt"
e4a = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(AI)Some people believe that success in life comes from taking risks or chances_.txt"
e5a = read_paragraph(file_path)

In [83]:
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)A person you know is planning to move to your town or city._.txt"
e1h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Businesses should hire employees for their entire lives._.txt"
e2h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Do you agree or disagree with the following statement_ Telephones.txt"
e3h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Do you agree or disagree with the following statement_.txt"
e4h = read_paragraph(file_path)
file_path = "/content/drive/Shareddrives/Comp440/articles txt version/Language Proficiency Exam/(Human)Some people believe that success in life comes from taking risks or chances._.txt"
e5h = read_paragraph(file_path)

#### First Paragraph

##### Store Paragraphs to List

In [84]:
first_paragraphs_a_e = [e1a[0],e2a[0],e3a[0],e4a[0],e5a[0]]

In [85]:
first_paragraphs_h_e = [e1h[0],e2h[0],e3h[0],e4h[0],e5h[0]]

##### Creating Cosine Similarity Dataset

In [86]:
df_e_first_a=get_cs_sim(first_paragraphs_a_e)

In [87]:
df_e_first_a["generator"] = "ai"
df_e_first_a["prompt"] = "exam"

In [88]:
df_e_first_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,exam
1,1,2,0.844017,ai,exam
2,1,3,0.817356,ai,exam
3,1,4,0.770848,ai,exam
4,1,5,0.842734,ai,exam
5,2,1,0.844017,ai,exam
6,2,2,1.000000,ai,exam
7,2,3,0.876157,ai,exam
8,2,4,0.903829,ai,exam
9,2,5,0.916162,ai,exam


In [89]:
df_e_first_h=get_cs_sim(first_paragraphs_h_e)

In [90]:
df_e_first_h["generator"] = "human"
df_e_first_h["prompt"] = "exam"

In [91]:
df_e_first_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,exam
1,1,2,0.801222,human,exam
2,1,3,0.773471,human,exam
3,1,4,0.770163,human,exam
4,1,5,0.785197,human,exam
5,2,1,0.801222,human,exam
6,2,2,1.000000,human,exam
7,2,3,0.790473,human,exam
8,2,4,0.812546,human,exam
9,2,5,0.877707,human,exam


#### Last Paragraph

##### Store Paragraphs to List

In [92]:
last_paragraphs_a_e = [e1a[len(e1a)-1],e2a[len(e2a)-1],e3a[len(e3a)-1],e4a[len(e4a)-1],e5a[len(e5a)-1]]

In [93]:
last_paragraphs_h_e = [e1h[len(e1h)-1],e2h[len(e2h)-1],e3h[len(e3h)-1],e4h[len(e4h)-1],e5h[len(e5h)-1]]

##### Creating Cosine Similarity Dataset

In [94]:
df_e_last_a=get_cs_sim(last_paragraphs_a_e)

In [95]:
df_e_last_a["generator"] = "ai"
df_e_last_a["prompt"] = "exam"

In [96]:
df_e_last_a

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,ai,exam
1,1,2,0.809686,ai,exam
2,1,3,0.801167,ai,exam
3,1,4,0.841978,ai,exam
4,1,5,0.810548,ai,exam
5,2,1,0.809686,ai,exam
6,2,2,1.000000,ai,exam
7,2,3,0.873148,ai,exam
8,2,4,0.887080,ai,exam
9,2,5,0.876640,ai,exam


In [97]:
df_e_last_h=get_cs_sim(last_paragraphs_h_e)

In [98]:
df_e_last_h["generator"] = "human"
df_e_last_h["prompt"] = "exam"

In [99]:
df_e_last_h

,Articl1,Article2,Cosine Similarity,generator,prompt
0,1,1,1.000000,human,exam
1,1,2,0.733483,human,exam
2,1,3,0.700581,human,exam
3,1,4,0.683598,human,exam
4,1,5,0.752661,human,exam
5,2,1,0.733483,human,exam
6,2,2,1.000000,human,exam
7,2,3,0.799581,human,exam
8,2,4,0.776057,human,exam
9,2,5,0.826074,human,exam


### Creating One Large Dataset for Cosine Similarity

In [100]:
BERT_cos_sin_first = pd.concat([df_n_first_h,df_n_first_a,df_b_first_a,df_b_first_h,df_e_first_a,df_e_first_h])
BERT_cos_sin_first["Paragraph Order"]="first"
BERT_cos_sin_first

,Articl1,Article2,Cosine Similarity,generator,prompt,Paragraph Order
0,1,1,1.000000,human,news,first
1,1,2,0.634030,human,news,first
2,1,3,0.668836,human,news,first
3,1,4,0.618219,human,news,first
4,1,5,0.659784,human,news,first
...,...,...,...,...,...,...
20,5,1,0.785197,human,exam,first
21,5,2,0.877707,human,exam,first
22,5,3,0.823279,human,exam,first
23,5,4,0.812321,human,exam,first


In [101]:
BERT_cos_sin_last = pd.concat([df_n_last_h,df_n_last_a,df_b_last_a,df_b_last_h,df_e_last_a,df_e_last_h])
BERT_cos_sin_last["Paragraph Order"]="last"
BERT_cos_sin_last

,Articl1,Article2,Cosine Similarity,generator,prompt,Paragraph Order
0,1,1,1.000000,human,news,last
1,1,2,0.825249,human,news,last
2,1,3,0.747916,human,news,last
3,1,4,0.752782,human,news,last
4,1,5,0.572435,human,news,last
...,...,...,...,...,...,...
20,5,1,0.752661,human,exam,last
21,5,2,0.826074,human,exam,last
22,5,3,0.779592,human,exam,last
23,5,4,0.731249,human,exam,last


In [102]:
BERT_cos_sin = pd.concat([BERT_cos_sin_first,BERT_cos_sin_last])
BERT_cos_sin

,Articl1,Article2,Cosine Similarity,generator,prompt,Paragraph Order
0,1,1,1.000000,human,news,first
1,1,2,0.634030,human,news,first
2,1,3,0.668836,human,news,first
3,1,4,0.618219,human,news,first
4,1,5,0.659784,human,news,first
...,...,...,...,...,...,...
20,5,1,0.752661,human,exam,last
21,5,2,0.826074,human,exam,last
22,5,3,0.779592,human,exam,last
23,5,4,0.731249,human,exam,last


In [103]:
BERT_cos_sin.to_csv('BERT_cos_sin.csv', index=False)

In [104]:
#from google.colab import files
#files.download('BERT_cos_sin.csv')

### Reading Cosine Similarity Dataset

In [105]:
# read the dataset from the stored csv file
BERT_cos_sin = pd.read_csv('/content/drive/Shareddrives/Comp440/articles txt version/BERT_cos_sin.csv')
BERT_cos_sin

,Articl1,Article2,Cosine Similarity,generator,prompt,Paragraph Order
0,1,1,1.000000,human,news,first
1,1,2,0.634030,human,news,first
2,1,3,0.668836,human,news,first
3,1,4,0.618220,human,news,first
4,1,5,0.659784,human,news,first
...,...,...,...,...,...,...
295,5,1,0.752661,human,exam,last
296,5,2,0.826074,human,exam,last
297,5,3,0.779592,human,exam,last
298,5,4,0.731249,human,exam,last
